In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
cwd = os.getcwd()
file_name = "MINDsmall_train/news.tsv"
file_path =  os.path.join(cwd, file_name)

In [3]:
news_df = pd.read_csv(file_path, sep='\t',header=None, names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' , 'URL', 'TitleEntities', 'AbstractEntities'])

In [4]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [5]:
news_df.drop(columns=['URL', 'TitleEntities' ,'AbstractEntities' ], inplace=True)

In [6]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [7]:
news_df.isna().sum()

NewsID            0
Category          0
SubCategory       0
Title             0
Abstract       2666
dtype: int64

In [8]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [31]:
from pinecone import Pinecone, ServerlessSpec

INDEX_NAME = 'pc-index-news'
pinecone = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=384, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [32]:
# Confirm we indeed created our  index

pinecone.list_indexes().names()

['pc-index-news']

In [33]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
from sentence_transformers import SentenceTransformer


def get_embeddings(articles, model=SentenceTransformer('all-MiniLM-L6-v2')):
   return model.encode(articles)

In [14]:
import math

# Function to check if a value is NaN
def is_nan(value):
    try:
        return math.isnan(value)
    except:
        return False

In [34]:
from tqdm.auto import tqdm, trange

CHUNK_SIZE=200
TOTAL_ROWS=10000
progress_bar = tqdm(total= TOTAL_ROWS)
use_columns = [0,1,2,3,4]
namespace = 'NewsTitles'

chunks = pd.read_csv(file_path, sep='\t',header=None,usecols=use_columns , 
                     names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' ]
                     , chunksize=CHUNK_SIZE
                     ,nrows=TOTAL_ROWS 
                     )



chunk_num = 0
for chunk in chunks:
    titles = chunk['Title'].tolist()
    newsIds = chunk['NewsID'].tolist()
    cleaned_titles = [title if not is_nan(title) else "" for title in titles]
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(newsIds[i])
                , 'values':embeddings[i],
                'metadata':{'title':cleaned_titles[i]}} 
               for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        try:
            index.upsert(prepped , namespace=namespace)
        except Exception as e:
            print(f"Error upserting chunk {chunk_num}: {e}")
        prepped = []
    progress_bar.update(len(chunk))

#final
if prepped:
    try:
        index.upsert(prepped,namespace=namespace)
    except Exception as e:
        print(f"Error upserting final chunk: {e}")

100%|██████████| 10000/10000 [01:45<00:00, 89.79it/s]

In [35]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'NewsTitles': {'vector_count': 10000}},
 'total_vector_count': 10000}

In [40]:
def get_recommendations(pinecone_index, search_term, namespace = 'NewsTitles' ,top_k=10 ):
  embed = get_embeddings(search_term)
  res = pinecone_index.query(namespace = namespace ,vector=embed.tolist(), top_k=top_k, include_metadata=True )
  return res

In [42]:
reco = get_recommendations(index, 'royalty', 'NewsTitles')
for r in reco.matches:
    print(f'{r.score} : {r.id}-{r.metadata["title"]}')

0.575195551 : N7716-What Do Prince George & Princess Charlotte Know About Their Royal Roles?
0.56598568 : N42777-Prince George's Royal Life in Photos
0.510303736 : N3164-Queen's Speech in Parliament: Black Rod, hostage-taking and a gold stick
0.506915033 : N39208-21 Things You Never Knew About America's 'Royal' Families
0.503744 : N48912-The British Royal Family and Their Pets: Which Is Your Favorite?
0.502944469 : N55528-The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By
0.490994751 : N38397-50 Incredible Photos of the Swedish Royal Family Through the Years
0.485220969 : N21477-Everything You Need to Know About the Swedish Royal Family
0.482100427 : N54727-The surprising age differences between your favorite royal couples
0.458627045 : N47247-Royal engagement rings


**Part2 : User Data**

In [123]:
file_name = "MINDsmall_train/behaviors.tsv"
file_path =  os.path.join(cwd, file_name)
behavior_df = pd.read_csv(file_path, sep='\t',header=None, names=['ImpressionID', 'UserID', 'Time', 'History' , 'Impressions' ])
behavior_df.head()

,ImpressionID,UserID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852 N55833 N64467 N3142 N13912 N29802 N44462 N29948 N4486 N5398 N14761 N47020 N65112 N31699 N37159 N61101 N14761 N3433 N10438 N61355 N21164 N22976 N2511 N48390 N58224 N48742 N35458 N24611 N37509 N21773 N41011 N19041 N25785,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N22407-0 N14592-0 N17059-1 N33677-0 N7821-0 N6890-0
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384 N46616 N52500 N33164 N47289 N24233 N62058 N26378 N49475 N18870,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N59685-0 N23814-1 N23446-0 N64174-0 N11817-0 N60550-0 N48225-0 N45509-0 N56711-0 N46821-0 N48017-0 N8015-0 N5364-0 N48722-0 N55555-0 N37348-0 N40109-0 N59495-0 N36226-0 N38779-0 N47346-0 N48875-0 N10960-0 N29739-0 N50872-0 N50592-0 N13131-0 N3839-0 N12330-0 N47098-0 N51570-0
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 N29757 N31825 N51891,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N60408-0 N61497-0 N47412-0 N41220-0 N1940-0 N724-0 N11363-0 N261-0 N33883-0 N36807-0 N11967-0 N17896-0 N13486-0 N10413-0 N54274-0 N4247-0 N27497-0 N38512-0 N30253-0 N45389-0 N20015-0 N20678-0 N54003-0 N35850-0 N33261-0 N32010-0 N57426-0 N7419-0 N50023-0 N36446-0 N26940-0 N28495-0 N19318-0 N4936-0 N28414-0 N25108-0 N32791-0 N23563-0 N39317-0 N16166-0 N37058-0 N64851-0 N46992-0 N57327-0 N12995-0 N58363-0 N53084-0 N11094-0 N36436-0 N305-0 N58241-0 N33212-0 N6975-0 N58114-0 N3344-0 N25406-0 N4741-0 N33885-0 N20915-0 N44941-0 N57319-0 N36532-0 N61822-0 N20527-0


In [135]:
behavior_df[behavior_df['History'].isna()]

,ImpressionID,UserID,Time,History,Impressions
28,29,U33207,11/11/2019 11:09:14 AM,NaN,N62212-0 N27521-0 N28983-0 N12028-0 N25437-0 N26262-0 N7041-1 N30475-0 N40065-0 N18708-0 N50406-0 N40530-0 N56598-0 N28324-0 N59981-0 N19735-0 N33848-0 N2823-1 N24686-0 N33872-0 N45456-0 N45076-0 N6056-1 N3894-0 N36545-0 N21915-0 N61359-0 N61623-0 N15761-0 N61549-0 N16804-0 N55204-0 N47433-0 N32791-0 N24180-0 N2869-0 N41698-0 N12800-0 N43338-0 N9139-0 N2-0 N5905-0 N41651-0 N6379-0 N45660-0 N28578-0 N45616-0 N3697-0 N58030-0 N8957-0 N17011-0 N3541-0 N54482-0 N27079-0 N21519-0 N48890-0 N49879-0 N26130-0 N41881-1 N38311-0 N23935-0 N1019-0 N18870-0 N51241-0 N43432-0 N23784-0 N47006-0 N47067-0 N63817-0 N4815-0 N47981-0 N15020-0 N47229-0 N62093-0 N60162-0 N1914-0 N11821-0 N13816-0 N6099-0 N30135-0 N55940-0 N30108-0 N33831-0 N23485-0 N4633-0 N43748-0 N31763-0 N51346-0 N11830-0 N3474-0 N36712-0 N7128-0 N36273-0 N53008-0 N37814-0 N21840-0 N10754-0 N59436-0 N17140-0 N57005-0 N5075-0 N27577-0 N64397-0 N8061-0 N40431-0 N6375-0 N20018-0 N14780-0 N25953-0 N47747-0 N36442-0 N47475-0 N17340-0 N11301-0 N43789-0 N4936-0 N53214-0 N44324-0 N36933-0 N7466-0 N5370-0 N13730-0 N24301-0 N64243-0 N21707-0 N35671-0 N42144-0 N2952-0 N64009-0 N54868-0 N63538-0 N35729-0 N22442-0 N26286-0 N43415-0 N50005-0 N36621-0 N59385-0 N40318-0 N48476-0 N10464-0 N48578-0 N26331-0 N13259-0 N17106-0 N52622-0 N6342-0 N42961-0 N58410-1 N55689-0 N29128-0 N17482-0 N54988-0 N22457-0 N29001-0 N38947-0 N46878-0 N26712-0 N49240-0 N58553-0 N63550-0 N52585-1 N23254-0 N24176-0 N57030-0 N26902-0 N61774-0 N15855-0 N59343-0 N53585-1 N13341-0 N3841-0 N46845-0 N40725-0
96,97,U89839,11/11/2019 4:57:51 AM,NaN,N12042-0 N35729-0 N49685-0 N62729-1
155,156,U5598,11/13/2019 2:01:40 PM,NaN,N7121-0 N37437-0 N53861-0 N45891-0 N21141-0 N50415-0 N14029-0 N30035-0 N4510-1 N26376-0 N35958-0 N41172-0 N22417-0 N17117-0 N64096-0 N13907-0 N62716-0 N21128-0 N38215-0 N8509-0 N55949-0 N47061-0 N35273-0 N4642-0 N11769-0 N7618-0 N43301-0 N58114-0 N21420-0 N15810-0 N2578-0 N38808-0 N55281-0 N56214-0 N45266-0 N36221-0
194,195,U19208,11/11/2019 3:57:34 PM,NaN,N23414-1 N52474-0 N6099-0 N5980-0 N1914-0
257,258,U6879,11/14/2019 5:33:52 PM,NaN,N9284-0 N23391-0 N42849-0 N33828-0 N57081-0 N27737-0 N54792-0 N17031-0 N52867-1 N55275-0 N6645-0 N16118-0 N5981-0 N6051-0 N7494-0 N50484-0 N41934-0 N55949-0 N15574-0 N63913-0 N21571-0 N1031-0 N16120-0 N36931-0 N28072-0 N36332-0 N64919-0 N60539-0 N20793-0 N62992-0 N19050-0 N11681-0 N64459-0 N29735-0 N55322-0 N21077-0 N61233-0 N29490-0 N3380-0 N43277-0 N45428-0 N48487-0 N59841-0 N19019-0 N30689-0 N14478-0 N36288-0 N23767-0 N18529-0 N15678-0 N53630-0 N34668-0 N14285-0 N37204-0 N47652-0 N8373-0 N61740-0 N12300-0 N37994-0 N22975-0 N34915-0 N45523-0 N1204-0 N47612-0 N6388-0 N42515-0 N53031-0 N14056-0 N58171-0 N56211-0 N4485-0 N6837-0 N18644-0 N22257-0 N54752-0 N63342-0 N41658-0 N9621-0 N8468-0 N11931-0 N36779-0 N54321-0 N48707-0
...,...,...,...,...,...
156850,156851,U85936,11/10/2019 8:52:02 AM,NaN,N41140-1 N4148-0
156890,156891,U4828,11/11/2019 9:41:21 AM,NaN,N8957-0 N60162-0 N16804-0 N24180-0 N31370-0 N22999-0 N48890-0 N40318-0 N26286-0 N43338-0 N59436-0 N35729-1 N15761-0 N29001-0 N42961-0 N64787-0 N8061-0 N19735-0 N33872-0 N58553-0 N50570-0 N24686-0 N27521-0 N29128-0 N40530-0 N10464-0 N32791-0 N49879-0 N33848-0 N38947-0 N154-0 N61623-0 N46845-0 N61359-1 N30135-0 N2869-0 N48476-0 N36621-0 N25953-0 N10754-0 N5075-0 N18378-0 N26376-0 N40431-0 N51346-0 N40065-0 N8399-0 N6099-0 N6379-0 N45375-0 N36442-0 N56598-0 N62212-0 N43789-0 N36712-0 N5370-0 N47229-0 N6342-0 N26331-0 N64243-0 N55204-0 N13259-0 N34390-0 N7128-0 N59981-0 N18708-0 N55689-0 N31727-0 N58710-0 N33159-0 N12523-0 N7809-0 N49240-0 N54988-0 N9271-0 N15435-0 N17140-0 N13801-0 N52622-0 N21707-0 N47067-0 N64542-0 N11830-0 N16342-0 N47981-0 N1914-0 N15855-0 N45076-0 N24176-0 N63538-0 N63550-0 N11821-0 N54482-0 N35776-0 N61214-0 N21915-0 N26043-0 N12800-0 N43415-0 N51639-1 N18870-0 N15830-0
156897,156898,U38290,11/14/2019 3:50:07 PM,NaN,N59267

In [125]:
len(behavior_df)

156965

In [133]:
behavior_df[behavior_df['UserID'].map(behavior_df['UserID'].value_counts()) >1].sort_values(by = ['UserID','ImpressionID'])

,ImpressionID,UserID,Time,History,Impressions
46639,46640,U1000,11/14/2019 1:00:56 AM,N29641 N1789 N41244,N16148-0 N43628-0 N13094-0 N32774-0 N57034-0 N29739-1 N39399-0 N55949-0 N37027-0 N49712-0 N14436-0 N47346-0 N55555-0 N55976-0 N20793-0 N50601-0 N5146-0 N18522-0 N59138-0 N7670-1 N55488-0 N8015-0 N614-0 N33831-0 N25295-0 N55275-0 N43301-0 N36016-0 N40130-0 N48875-0 N41698-0 N56711-0 N45509-0 N22257-0 N7121-0 N11817-0 N41612-0 N39187-0 N16618-0 N59214-0 N60161-0 N55281-0 N10960-0 N8029-0 N43487-0 N35387-0 N9997-0 N39005-0 N42457-0 N64174-0 N63174-0 N16282-0 N58660-0 N44249-0 N1012-0 N6216-0 N15719-0 N20576-0 N16844-0 N3957-0 N50872-0 N36261-0 N20770-0 N41122-0 N7618-0 N10552-0 N29517-0 N61787-0 N53875-0 N37194-0 N64412-0 N30713-0 N17759-0 N20036-0 N19867-0 N23805-0 N12124-0 N3031-0 N3841-0 N50014-0 N10616-0 N62894-0 N5652-0 N4021-0 N8595-0 N21701-0 N26233-0 N5048-0 N56214-0 N3167-0 N23272-0 N54752-0
74344,74345,U1000,11/14/2019 10:37:21 PM,N29641 N1789 N41244,N14478-0 N58264-0 N7494-0 N46917-0 N22257-0 N11378-0 N2235-0 N28072-0 N3841-0 N64459-0 N47652-0 N30290-0 N41934-0 N50055-0 N31552-0 N22978-0 N8595-0 N61697-0 N29490-0 N57007-0 N64037-0 N6837-0 N55712-0 N27289-0 N39369-0 N48707-0 N2960-0 N21571-0 N7852-0 N31141-0 N39683-0 N57327-0 N48487-0 N12446-0 N26063-0 N47280-0 N10051-0 N50484-0 N55322-0 N60747-0 N61829-0 N22975-0 N58081-0 N54792-0 N63913-0 N54803-0 N42950-0 N56211-0 N23767-0 N398-0 N37233-0 N483-0 N61233-0 N27737-0 N58656-1 N34930-0 N42515-0
86766,86767,U1000,11/13/2019 11:16:18 PM,N29641 N1789 N41244,N35273-0 N7618-0 N55281-0 N4021-0 N16148-0 N3841-0 N54752-0 N61787-0 N1012-0 N57034-0 N55976-0 N59138-0 N27869-0 N21128-0 N5048-0 N28767-0 N48875-0 N49712-0 N4642-0 N16282-0 N18522-0 N13094-0 N7121-0 N37194-0 N33831-0 N36252-0 N14436-0 N64174-0 N41698-0 N53875-1 N8015-0 N23272-0 N35387-0 N16844-0 N55949-0 N39399-0 N3957-0 N32774-0 N36261-0 N3031-0 N42457-0 N37027-0 N5652-0 N47624-0 N41122-0 N55555-0 N9997-0 N51287-0 N40130-0 N16618-0 N14029-0 N56214-0 N58660-0 N62894-0 N38053-0 N8029-0
60202,60203,U10001,11/14/2019 5:46:54 AM,N47937 N51706 N56447 N61319 N27644 N18030 N27256 N33976 N34562 N2735 N20639 N22816 N56814 N46444 N59380,N42860-0 N62750-0 N33025-0 N3167-0 N51975-0 N40109-0 N7670-0 N25324-0 N39187-0 N58660-0 N5597-0 N43487-0 N32854-0 N50288-0 N7618-0 N11817-0 N56711-0 N36016-0 N19776-0 N3957-0 N7121-0 N26673-0 N50014-0 N8791-0 N9009-0 N19915-0 N58814-0 N1031-1 N46923-0 N16844-0 N54752-0 N30071-0 N16419-0 N35387-0 N46186-0 N16777-0 N38442-0 N63174-0 N59685-0 N10833-1 N30089-0 N64174-0 N10960-0 N5652-0 N54023-0 N50872-0 N17031-0 N55555-0 N60272-0 N18522-0 N42457-0 N31406-0 N5146-0 N9135-0 N14436-0 N29739-0 N17759-0 N45509-0 N47098-0 N51970-0 N27638-0 N3894-0 N46066-0 N6664-0 N60550-0 N27869-0 N38695-0 N16439-0 N8595-0 N20576-0 N43301-0 N8015-0 N11940-0 N59138-0 N48875-0 N37854-0
155922,155923,U10001,11/11/2019 5:30:21 AM,N47937 N51706 N56447 N61319 N27644 N18030 N27256 N33976 N34562 N2735 N20639 N22816 N56814 N46444 N59380,N35729-0 N56598-0 N48759-0 N49685-0 N33632-0 N33848-0 N35937-1 N27581-0
...,...,...,...,...,...
76920,76921,U9996,11/13/2019 10:07:24 AM,N28296 N4719 N60340 N31165 N8448,N287-1 N64094-0 N51048-0 N39010-0
120621,120622,U9996,11/14/2019 8:41:12 AM,N28296 N4719 N60340 N31165 N8448,N38779-0 N1539-0 N45523-0 N40109-0 N50872-0 N29212-0 N23877-0 N6477-0 N47098-1
154397,154398,U9996,11/14/2019 8:40:37 AM,N28296 N4719 N60340 N31165 N8448,N40109-0 N6477-0 N23877-0 N23446-1 N50872-0 N38779-0
62952,62953,U9997,11/13/2019 2:41:42 PM,N46759 N46990 N9072 N11727 N50744 N38367 N33333 N55285 N90 N64836 N64459 N18106 N11929,N35273-0 N4510-0 N48410-1 N29170-0 N26262-0 N57151-0 N7618-0 N8643-0 N56214-0 N22417-0 N35047-0 N14029-0 N36261-0 N51048-0 N36221-0 N38215-0 N17117-0 N37437-0 N47061-0 N13907-0 N8509-0


In [136]:
behavior_df = behavior_df.dropna(subset=['History'])

In [137]:
behavior_df['History'].isna().sum()

0

In [138]:
len(behavior_df)

153727

In [139]:
#split NewsID in history into separate rows :
split_history_df = behavior_df['History'].str.split(' ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='NewsID')

In [140]:
split_history_df.head()

,index,NewsID
0,0,N55189
1,0,N42782
2,0,N34694
3,0,N45794
4,0,N18445


In [144]:
expanded_df = behavior_df.merge(split_history_df, left_index=True, right_on='index').drop(columns=['index'])
expanded_df.head(10)


,ImpressionID,UserID,Time,History,Impressions,NewsID
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N55189
1,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N42782
2,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N34694
3,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N45794
4,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N18445
5,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N63302
6,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N10414
7,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N19347
8,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N31801
9,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852 N55833 N64467 N3142 N13912 N29802 N44462 N29948 N4486 N5398 N14761 N47020 N65112 N31699 N37159 N61101 N14761 N3433 N10438 N61355 N21164 N22976 N2511 N48390 N58224 N48742 N35458 N24611 N37509 N21773 N41011 N19041 N25785,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N22407-0 N14592-0 N17059-1 N33677-0 N7821-0 N6890-0,N31739


In [145]:
len(expanded_df)

5107639

In [150]:
expanded_df['History'].isna().sum()

0

In [152]:
df_merged = pd.merge(expanded_df, news_df, on='NewsID', how='left')
df_merged.head(10)

,ImpressionID,UserID,Time,History,Impressions,NewsID,Category,SubCategory,Title,Abstract
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, Off The Rails Introduction","We'd like to solve the puzzle, Pat: Blair Davis' loveless marriage? On Monday, ""Wheel of Fortune"" welcomed as a new contestant trucking business owner Blair Davis, who offered a biting introduction for himself. When host Pat Sajak asked the man from Cardiff, California, about his family, Davis plunged into one of the darkest personal summaries the show has likely ever heard. ""I've been trapped in a loveless marriage for the last 12 years to an..."
1,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 victory over the Astros,The Yankees kept hope alive thanks to some impressive starting pitching and a pair of early home runs.
2,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to Speaking to People' Right Now,Rosie O'Donnell: Barbara Walters Isn't 'Up to Speaking to People' Right Now
3,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N45794,news,newscrime,"Four flight attendants were arrested in Miami's airport after bringing in thousands in cash, police say","Four American Airlines flight attendants were arrested at the Miami International Airport and charged with money laundering after bringing large amounts of cash into the country, police said."
4,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as series goes on hold,"Parting is such sweet sorrow, say the Wolverines."
5,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N63302,lifestyle,lifestylebuzz,This Wedding Photo of a Canine Best Man Captures Just How Deep a Dog's Love Truly Is,"When Mark Doublet made his dog, Marley, the best man at his wedding, he took his duties very seriously, and a photo of them together has gone viral."
6,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N10414,movies,movienews,"Robert Evans, 'Chinatown' Producer and Paramount Chief, Dies at 89","Robert Evans, the Paramount executive who produced ""Chinatown"" and ""Urban Cowboy"" and whose life became as melodramatic and jaw-dropping as any of his films, died on Saturday night. He was 89. Even though Hollywood history is filled with colorful characters, few can match the tale of Evans, whose life would seem far-fetched if it were"
7,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N19347,news,newspolitics,Former US Senator Kay Hagan dead at 66,"Former U.S. Sen. Kay Hagan, a one-time Capitol Hill intern who went on to become North Carolina's first Democratic female senator, died Monday at her Greensboro home. She was 66."
8,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801,N55689-1 N35729-0,N31801,news,newspolitics,Joe Biden reportedly denied Communion at a South Carolina church because of his stance on abortion,Joe Biden has a complicated history with the Catholic Church.
9,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852

In [153]:
len(df_merged)

5107639

In [155]:
pd.set_option('display.max_colwidth', None)
df_concat = df_merged.groupby('ImpressionID')['Title'].apply(lambda x: ';'.join(x)).reset_index()
df_concat = df_concat.rename(columns={'Title': 'Titles'})
df_concat.head(10)

ImpressionID  \
0             1   
1             2   
2             3   
3             4   
4             5   
5             6   
6             7   
7             8   
8             9   
9            10   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [156]:
len(df_concat)

153727

In [159]:
final_user_df = pd.merge(behavior_df , df_concat, on="ImpressionID", how='inner')
final_user_df.head()

ImpressionID  UserID                   Time  \
0             1  U13740  11/11/2019 9:05:58 AM   
1             2  U91836  11/12/2019 6:11:30 PM   
2             3  U73700  11/14/2019 7:01:48 AM   
3             4  U34670  11/11/2019 5:28:05 AM   
4             5   U8125  11/12/2019 4:11:21 PM   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         History  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 N55189 N42782 N34694 N45794 N18445 N63302 N10414 N19347 N31801   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129 N1569 N17686 N13008 N21623 N6233 N14340 N48031 N62285 N44383 N23061 N16290 N6244 N45099 N58715 N59049 N7023 N50528 N42704 N46082 N8275 N15710 N59026 N8429 N30867 N56514 N19709 N31402 N31741 N54889 N9798 N62612 N2663 N16617 N6087 N13231 N63317 N61388 N59359 N51163 N30698 N34567 N54225 N32852 N55833 N64467 N3142 N13912 N29802 N44462 N29948 N4486 N5398 N14761 N47020 N65112 N31699 N37159 N61101 N14761 N3433 N10438 N61355 N21164 N22976 N2511 N48390 N58224 N48742 N35458 N24611 N37509 N21773 N41011 N19041 N25785   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                  N10732 N25792 N7563 N21087 N41087 N5445 N60384 N46616 N52500 N33164 N47289 N24233 N62058 N26378 N49475 N18870   
3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             N45729 N2203 N871 N53880 N41375 N43142 N33013 N29757 N31825 N51891   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    N10078 N56514 N14904 N33740   

                                                                                                                                                                                                                                                                                                                             

In [160]:
final_user_df['Titles'].isna().sum()

0

In [161]:
final_user_df.dtypes

ImpressionID     int64
UserID          object
Time            object
History         object
Impressions     object
Titles          object
dtype: object

In [162]:
final_user_df['Titles'] = final_df['Titles'].astype('string')

In [52]:
CHUNK_SIZE = 200
N_ROWS = 1000
prepped = []
progress_bar = tqdm(total= final_df.shape[0])

def chunk_df(df ,CHUNK_SIZE ):
    for i in range(0,len(df), CHUNK_SIZE): #for i in range(0,len(df), CHUNK_SIZE)
        yield df.iloc[i:i+CHUNK_SIZE]

chunk_num = 0
for chunk in chunk_df(final_df.head(N_ROWS) , CHUNK_SIZE):
    titles = chunk['Titles'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(chunk_num*CHUNK_SIZE+i), 'values':embeddings[i],
                'metadata':{'UserID':chunk['UserID'].iloc[i]}} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        try:
            index.upsert(prepped)
        except:
            print(f"Error upserting chunk {chunk_num}: {e}")
        prepped = []
    progress_bar.update(len(chunk))

#final
if prepped:
    try:
        index.upsert(prepped)
    except Exception as e:
        print(f"Error upserting final chunk: {e}")


 98%|█████████▊| 153600/156965 [2:33:37<02:40, 20.92it/s] 

TypeError: 'NAType' object is not subscriptable